In [47]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from keras import backend as K
K.set_image_dim_ordering('th')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam

In [48]:
PATH=os.getcwd()
data_path= PATH + '\data'

data_dir_list=os.listdir(data_path)
data_dir_list

['Driver_No_Phone', 'Driver_Phone']

In [49]:
img_rows=128
img_cols=128
num_channel=1
num_epoch=10

num_classes = 2
img_data_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    #print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

(200, 128, 128)


In [50]:
if num_channel==1:
    if K.image_dim_ordering()=='th':
        img_data= np.expand_dims(img_data,axis=1) 
        print (img_data.shape)
    else:
        img_data= np.expand_dims(img_data, axis=4) 
        print (img_data.shape)

else:
    if K.image_dim_ordering()=='th':
        img_data=np.rollaxis(img_data,3,axis=1)
        print (img_data.shape)

(200, 1, 128, 128)


In [51]:
USE_SKLEARN_PREPROCESSING=True
if USE_SKLEARN_PREPROCESSING:
	# using sklearn for preprocessing
	from sklearn import preprocessing
	
	def image_to_feature_vector(image, size=(128, 128)):
		# resize the image to a fixed size, then flatten the image into
		# a list of raw pixel intensities
		return cv2.resize(image, size).flatten()
	
	img_data_list=[]
	for dataset in data_dir_list:
		img_list=os.listdir(data_path+'/'+ dataset)
		print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
		for img in img_list:
			input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
			input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
			input_img_flatten=image_to_feature_vector(input_img,(128,128))
			img_data_list.append(input_img_flatten)
	
	img_data = np.array(img_data_list)
	img_data = img_data.astype('float32')
	print (img_data.shape)
	img_data_scaled = preprocessing.scale(img_data)
	print (img_data_scaled.shape)
	
	print (np.mean(img_data_scaled))
	print (np.std(img_data_scaled))
	
	print (img_data_scaled.mean(axis=0))
	print (img_data_scaled.std(axis=0))
	
	if K.image_dim_ordering()=='th':
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],num_channel,img_rows,img_cols)
		print (img_data_scaled.shape)
		
	else:
		img_data_scaled=img_data_scaled.reshape(img_data.shape[0],img_rows,img_cols,num_channel)
		print (img_data_scaled.shape)

Loaded the images of dataset-Driver_No_Phone

Loaded the images of dataset-Driver_Phone

(200, 16384)
(200, 16384)
-6.4074994e-09
0.9999999
[ 6.5565109e-09 -1.1622905e-08 -1.4305114e-08 ... -1.8700957e-08
  2.4288893e-08 -4.4889750e-09]
[1.         0.9999998  0.99999976 ... 1.         0.9999998  0.9999996 ]
(200, 1, 128, 128)


C:\Users\karan\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\karan\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:177: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [69]:
if USE_SKLEARN_PREPROCESSING:
	img_data=img_data_scaled
num_classes = 2

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:100]=0
labels[100:]=1

names = ['Driver_No_Phone', 'Driver_Phone']
	  
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
input_shape=img_data[0].shape


In [53]:
model = Sequential()

model.add(Convolution2D(32, 3,3,border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=["accuracy"])
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])

# Viewing model_configuration

model.summary()

C:\Users\karan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(1, 128, 1..., padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\karan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  """
C:\Users\karan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\karan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  if sys.path[0] == '':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 128, 128)      320       
_________________________________________________________________
activation_25 (Activation)   (None, 32, 128, 128)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 126, 126)      9248      
_________________________________________________________________
activation_26 (Activation)   (None, 32, 126, 126)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 32, 63, 63)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 32, 63, 63)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 61, 61)        18496     
__________

In [54]:
hist = model.fit(X_train, y_train, batch_size=16, nb_epoch=num_epoch, verbose=1, validation_data=(X_test, y_test))

C:\Users\karan\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 61s 382ms/step - loss: 0.7326 - acc: 0.5375 - val_loss: 0.7052 - val_acc: 0.3500
Epoch 2/10
160/160 [==============================] - 62s 384ms/step - loss: 0.6874 - acc: 0.5312 - val_loss: 0.7025 - val_acc: 0.3500
Epoch 3/10
160/160 [==============================] - 58s 366ms/step - loss: 0.6793 - acc: 0.5500 - val_loss: 0.7021 - val_acc: 0.3500
Epoch 4/10
160/160 [==============================] - 70s 438ms/step - loss: 0.6595 - acc: 0.5750 - val_loss: 0.6958 - val_acc: 0.5750
Epoch 5/10
160/160 [==============================] - 63s 393ms/step - loss: 0.6633 - acc: 0.6250 - val_loss: 0.6909 - val_acc: 0.5250
Epoch 6/10
160/160 [==============================] - 55s 345ms/step - loss: 0.6471 - acc: 0.6688 - val_loss: 0.7035 - val_acc: 0.4750
Epoch 7/10
160/160 [==============================] - 56s 351ms/step - loss: 0.6173 - acc: 0.6500 - val_loss: 0.6964 - val_acc: 0.5250
Epoch 8/10

In [75]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 0.69507896900177
Test accuracy: 0.55


In [81]:
test_image2 = X_test[7:8]
print (test_image2.shape)

(1, 1, 128, 128)


In [82]:
print(model.predict(test_image2))
print(model.predict_classes(test_image2))
print(y_test[7:8])

[[0.43186656 0.5681334 ]]
[1]
[[1. 0.]]


In [87]:
from sklearn.metrics import classification_report,confusion_matrix
import itertools

Y_pred = model.predict(X_test)
print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
y_pred = model.predict_classes(X_test)
print(y_pred)
target_names = ['class 0(N)', 'class 1(Y)']
					


print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

[[0.40900558 0.59099436]
 [0.44040525 0.55959475]
 [0.41876078 0.5812392 ]
 [0.52174246 0.47825754]
 [0.6390658  0.3609342 ]
 [0.40662867 0.59337133]
 [0.4759207  0.52407926]
 [0.43186656 0.5681335 ]
 [0.5116017  0.48839834]
 [0.42639592 0.57360405]
 [0.48393998 0.51606005]
 [0.4498119  0.5501881 ]
 [0.3635201  0.6364799 ]
 [0.44023862 0.55976135]
 [0.53283286 0.46716717]
 [0.33959848 0.6604015 ]
 [0.43073428 0.56926566]
 [0.52070534 0.4792946 ]
 [0.5999756  0.40002444]
 [0.63721687 0.36278313]
 [0.6524456  0.3475544 ]
 [0.55770004 0.44229996]
 [0.6201233  0.3798766 ]
 [0.56591743 0.43408257]
 [0.5354308  0.46456924]
 [0.53317106 0.46682897]
 [0.6391698  0.3608302 ]
 [0.53468007 0.46531993]
 [0.36319655 0.63680345]
 [0.57171005 0.42828992]
 [0.58625674 0.4137432 ]
 [0.43100542 0.5689946 ]
 [0.53777826 0.46222183]
 [0.62571156 0.37428847]
 [0.6924742  0.3075258 ]
 [0.34177604 0.65822387]
 [0.5078661  0.4921339 ]
 [0.4621776  0.53782237]
 [0.56968755 0.43031242]
 [0.49814138 0.5018586 ]]